In [58]:
import vectorbtpro as vbt
import numpy as np
import pandas as pd

vbt.settings.set_theme('dark')
vbt.settings['plotting']['layout']['width'] = 800
vbt.settings['plotting']['layout']['height'] = 400

In [59]:
df = pd.read_csv('2ySOLdata1h.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df.set_index('timestamp', inplace=True)

In [60]:
data = vbt.Data.from_data(df)
data

In [62]:
open_price = data.get('Open')
high_price = data.get('High')
low_price = data.get('Low')
close_price = data.get('Close')

In [63]:
# vbt.phelp(vbt.MA.run)

In [65]:
adx = vbt.ADX.run(high_price, low_price, close_price, window=14)
atr = vbt.ATR.run(high_price, low_price, close_price, window=14)
bbands = vbt.BBANDS.run(close_price, window=14)
rsi = vbt.RSI.run(close_price)
sma = vbt.MA.run(close_price, window=20)
strend = vbt.SUPERTREND.run(high_price, low_price, close_price, period=7, multiplier=3)

data.data['symbol']['ADX'] = adx.adx
data.data['symbol']['ATR'] = atr.atr
data.data['symbol']['BBAND'] = bbands.bandwidth
data.data['symbol']['RSI'] = rsi.rsi
data.data['symbol']['SMA'] = sma.ma
data.data['symbol']['STREND'] = strend.trend
data.data['symbol'] = data.data['symbol'].dropna()
predictor_list = ['Close', 'ADX', 'ATR', 'BBAND', 'RSI', 'SMA', 'STREND']

X = data.data['symbol'][predictor_list]

y = data.data['symbol']['signal']

In [66]:
from sklearn.model_selection import train_test_split
split_percentage = 0.7
split = int(split_percentage*len(X))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1-split_percentage)


In [67]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion='gini', max_depth=10, min_samples_leaf=5, random_state=13).fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [68]:
y_pred_proba = clf.predict_proba(X_test)

In [69]:
y_pred_proba[:, 0]

array([0.28571429, 0.        , 0.04761905, ..., 0.        , 0.        ,
       0.        ])

In [70]:
# from sklearn.metrics import classification_report
# from sklearn.metrics import accuracy_score
# report = classification_report(y_test, y_pred)
# accuracy = accuracy_score(y_test, y_pred)
# print(accuracy)

In [71]:
# Store the test dataset in new dataframe df_split
df_split = data.data['symbol'][split:].copy()

# Store the decision tree's predicted output to signal column of df_split dataframe
df_split.loc[:, "signal short"] = y_pred_proba[:, 0]
df_split.loc[:, "signal none"] = y_pred_proba[:, 1]
df_split.loc[:, "signal long"] = y_pred_proba[:, 2]



In [72]:
signalshort = df_split['signal short']
signalnone = df_split['signal none']
signallong = df_split['signal long']
entries = signallong > 0.4
exits = signalshort > 0.8


In [73]:
pf = vbt.Portfolio.from_signals(
    close=df_split.Close, 
    long_entries=entries, 
    long_exits=exits,
    size=100,
    size_type='value',
    init_cash='auto'
)

In [74]:
# import joblib
# def save_model(model, model_filename='trained_model.joblib'):
#     joblib.dump(model, model_filename)
#     print(f'Model saved as {model_filename}')
# save_model(clf)

In [75]:
pf.plot(settings=dict(bm_returns=False)).show()

In [76]:
pf.stats()

Start                         2023-05-26 21:00:00+00:00
End                           2023-12-31 06:00:00+00:00
Period                                218 days 10:00:00
Start Value                                  106.000447
Min Value                                      85.84789
Max Value                                    281.540383
End Value                                    263.738825
Total Return [%]                             148.809163
Benchmark Return [%]                         421.028013
Total Time Exposure [%]                       74.132011
Max Gross Exposure [%]                            100.0
Max Drawdown [%]                              24.702998
Max Drawdown Duration                  98 days 03:00:00
Total Orders                                         39
Total Fees Paid                                     0.0
Total Trades                                         20
Win Rate [%]                                  63.157895
Best Trade [%]                                62